In [1]:
import pandas as pd
import numpy as np

from zipfile import ZipFile
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

c:\dev\jupyter\dist\data_science_env\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [8]:
with ZipFile('smsspamcollection.zip') as f:
    f.extractall('./data')

In [14]:
data = pd.read_csv('./data/SMSSpamCollection.txt', sep='\t', header=None, names=['target', 'text'])

In [15]:
!rd /s /q data

In [17]:
data['target'] = data['target'].map({'ham': 0, 'spam': 1})

In [19]:
targets = data['target'].values
texts = data['text'].values

In [20]:
targets

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [21]:
texts

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [22]:
vectorizer = CountVectorizer()
feature_matrix = vectorizer.fit_transform(texts)

In [26]:
from sklearn.metrics import make_scorer

In [31]:
model = LogisticRegression()
cross_val_score(model, feature_matrix, targets, cv=10, scoring=make_scorer(f1_score)).mean()

0.9311542822856882

In [32]:
model.fit(feature_matrix, targets)

LogisticRegression()

In [36]:
for sample in [
    ['FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB'],
    ['FreeMsg: Txt: claim your reward of 3 hours talk time'],
    ['Have you visited the last lecture on physics?'],
    ['Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$'],
    ['Only 99$']
]:
    x = vectorizer.transform(sample)
    print(model.predict(x))

[1]
[1]
[0]
[0]
[0]


In [38]:
for n_gram_range in [(2, 2), (3, 3), (1, 3)]:
    vectorizer_n_gram = CountVectorizer(ngram_range=n_gram_range)
    feature_matrix_n_gram = vectorizer_n_gram.fit_transform(texts)
    print(cross_val_score(model, feature_matrix_n_gram, targets, cv=10, scoring=make_scorer(f1_score)).mean())

0.8168951028736983
0.7249691484935827
0.9216545237836128


In [39]:
from sklearn.naive_bayes import MultinomialNB
bayes = MultinomialNB()
for n_gram_range in [(2, 2), (3, 3), (1, 3)]:
    vectorizer_n_gram = CountVectorizer(ngram_range=n_gram_range)
    feature_matrix_n_gram = vectorizer_n_gram.fit_transform(texts)
    print(cross_val_score(bayes, feature_matrix_n_gram, targets, cv=10, scoring=make_scorer(f1_score)).mean())

0.6457761003991855
0.37861105465217015
0.888454144419353


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_tf = TfidfVectorizer()
feature_matrix_tf = vectorizer_tf.fit_transform(texts)
cross_val_score(model, feature_matrix_tf, targets, cv=10, scoring=make_scorer(f1_score)).mean()

0.8520118540708351